In [1]:
from functools import reduce

from cycler import cycler
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numbers import Number
import numpy as np
import pandas as pd
#from scipy import stats as sstats
import wmfdata as wmf
from wmfdata.utils import pct_str, sig_figs
from wmfdata.charting import pct_fmt, M_fmt, comma_fmt

import gspread
from google.oauth2.service_account import Credentials

In [2]:
%matplotlib inline
wmf.charting.set_mpl_style()

In [3]:
FILE = "metrics/metrics.tsv"
metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

FILE_D =  "metrics/diversity_metrics.tsv"
diversity_metrics = pd.read_csv(FILE_D, sep="\t", parse_dates=["month"]).set_index("month")

graph_end = metrics.index[-1]
graph_start = graph_end - relativedelta(months = 47)

# Data transformation

In [4]:
# Transform content flows into content stocks
def stock_to_flow(arr):
    return arr[-1] - arr[0]

flow_cols = [
    "net_new_content_pages", 
    "net_new_Wikipedia_articles", 
    "net_new_Commons_content_pages", 
    "net_new_Wikidata_entities"
]
stock_cols = [
    "content_pages", 
    "Wikipedia_articles", 
    "Commons_content_pages",
    "Wikidata_entities"
]

name_map = {flow_cols[i]: stock_cols[i] for i in range(len(stock_cols))}

stock_metrics = metrics[flow_cols].cumsum().rename(name_map, axis=1)

metrics = pd.concat([metrics, stock_metrics], axis=1)

# Show results
metrics.tail()

,active_editors,anonymous_edits,global_south_active_editors,global_south_edits,global_south_new_editor_retention,global_south_nonbot_edits,mobile-heavy_wiki_active_editors,mobile-heavy_wiki_edits,mobile-heavy_wiki_new_editor_retention,mobile-heavy_wiki_nonbot_edits,...,other_nonbot_edits,returning_active_editors,revert_rate,total_edits,uploads,wikidata_edits,content_pages,Wikipedia_articles,Commons_content_pages,Wikidata_entities
month,,,,,,,,,,,,,,,,,,,,,
2021-04-01,97125.0,2178544.0,24252.0,3450291.0,0.051659,2604520.0,5465.0,1240569.0,NaN,632886.0,...,4772515.0,78295.0,0.051466,43880912.0,1761754.0,17784086.0,278627062.0,57585855.0,73381246.0,94901905.0
2021-05-01,98730.0,2294554.0,25308.0,3414303.0,0.056190,2700943.0,5560.0,1226707.0,NaN,641991.0,...,5280862.0,78854.0,0.053275,49624611.0,1270095.0,20602931.0,280854498.0,57827092.0,74599729.0,95223483.0
2021-06-01,91670.0,2089434.0,24610.0,3122716.0,0.057165,2530074.0,5103.0,963573.0,0.061350,567731.0,...,5218602.0,73947.0,0.055657,42163576.0,796256.0,18875684.0,282593964.0,58152941.0,75350440.0,95573046.0
2021-07-01,89307.0,2049005.0,25186.0,3184875.0,0.053920,2744544.0,5320.0,1144866.0,0.060699,598855.0,...,5250912.0,72910.0,0.053741,44780394.0,738459.0,18720677.0,284846085.0,58413725.0,76045398.0,95914464.0
2021-08-01,90312.0,2100177.0,25249.0,3594274.0,NaN,2735392.0,5606.0,1322700.0,0.058824,618020.0,...,5587077.0,74013.0,0.050318,47913055.0,1311357.0,19905473.0,287177993.0,58660786.0,77304305.0,96496720.0


In [5]:
metrics_all = pd.concat([metrics, diversity_metrics], axis=1)
metrics_all  = metrics_all.loc[:,~metrics_all.columns.duplicated()]


# Report

In [6]:
report_order = [
     #Editors
    'active_editors',
        'new_active_editors',
        'returning_active_editors',
    'new_editor_retention',
    # Content
    'content_pages', 
        'Wikipedia_articles',
        'Commons_content_pages', 
        'Wikidata_entities',
    'net_new_content_pages',
        'net_new_Wikipedia_articles',
        'net_new_Commons_content_pages',
        'net_new_Wikidata_entities',
    'revert_rate',
    'total_edits',
        'mobile_edits',
        'wikidata_edits',
        'uploads',
        'other_nonbot_edits',
        'anonymous_edits',
        'non_anonymous_edits',
    #Diversity Content
    'global_north_net_new_content',
    'global_south_net_new_content',
    'global_north_edits',
    'global_south_edits',
    #Diversity Editors
    'global_north_active_editors',
    'global_south_active_editors',
    'global_north_new_editor_retention',
    'global_south_new_editor_retention'

]

In [7]:
def fmt_num(x):
    if isinstance(x, Number) and not pd.isnull(x):
        x = sig_figs(x, 3)
        
        if x < 5:
            return pct_str(x)
        else:
            return "{:,.0f}".format(x)
    else:
        return x

def calc_rpt(ser):   
    cur = ser[-1]
    
    try:
        yr_prev = ser[-13]
        yoy_change = (cur / yr_prev) - 1
    except IndexError:
        yoy_change = None
        
    try:
        cagr = (ser[-1] / ser[-48])**(1/4) - 1
    except IndexError:
        cagr = None
        
    
    res = [cur, yoy_change, cagr]
    return pd.Series(
        [fmt_num(n) for n in res],
        index=["value", "yoy_change", "4_yr_cagr"]
    )

In [8]:
(
    metrics_all
    .apply(calc_rpt)
    .transpose()
    .reindex(report_order)
    .fillna("—")
)

,value,yoy_change,4_yr_cagr
active_editors,"90,300",1.2%,3.2%
new_active_editors,"16,300",-7.9%,-0.8%
returning_active_editors,"74,000",3.5%,4.2%
new_editor_retention,6.0%,8.9%,2.9%
content_pages,"287,000,000",10.6%,14.9%
Wikipedia_articles,"58,700,000",6.3%,6.0%
Commons_content_pages,"77,300,000",20.0%,16.3%
Wikidata_entities,"96,500,000",7.1%,27.4%
net_new_content_pages,"2,330,000",15.0%,-14.9%
net_new_Wikipedia_articles,"247,000",-14.0%,-11.0%
